Exploratory Data Analysis 🏗️📊
Configuration

In [ ]:
import pandas as pd  # Importa a biblioteca Pandas para manipulação de dados
import numpy as np  # Importa a biblioteca NumPy para operações numéricas
import sqlite3  # Importa a biblioteca SQLite para interação com bancos de dados SQLite
import dash  # Importa a biblioteca Dash para criação de aplicativos web interativos
import dash_table  # Importa o componente dash_table para exibir dados em tabelas
from dash import dcc, html  # Importa os componentes dcc (gráficos) e html do Dash
import plotly.express as px  # Importa a biblioteca Plotly Express para criar gráficos de forma fácil
import matplotlib.pyplot as plt  # Importa a biblioteca Matplotlib para criação de gráficos
import io  # Importa a biblioteca io para lidar com streams de dados
import base64  # Importa a biblioteca base64 para codificar imagens

conn = sqlite3.connect('G:/Meu Drive/Documents/GitHubPublished/DataScienceProject/database/ecommerceProject.db')  # Conecta ao banco de dados SQLite

# Define um dicionário com consultas SQL para diferentes análises
dictQuery = {
    "queryCustomers": 'SELECT * FROM customers LIMIT 5',
    "queryOrders": 'SELECT * FROM orders LIMIT 5',
    "queryProducts": 'SELECT * FROM products LIMIT 5',
    "queryCategories": 'SELECT * FROM categories LIMIT 5',
    "queryOrderItens": 'SELECT * FROM order_items LIMIT 5',
    "queryCountProducts": 'SELECT COUNT(*) AS total_products FROM products',
    "queryCountCategories": 'SELECT COUNT(*) AS total_categories FROM categories',
    "queryAvgSales": 'SELECT Round(AVG(order_value),2) AS avg_sales FROM orders',
    "queryDistinctCategories": 'SELECT DISTINCT category_name FROM categories',
    "queryProductsByCategories": '''
        SELECT c.category_name, COUNT(DISTINCT p.product_name) AS count
        FROM categories c
        JOIN products p ON c.category_id = p.category_id
        GROUP BY c.category_name
        ORDER BY c.category_name''',
    "queryTop10CustomerValue": '''
        SELECT c.customer_name, ROUND(SUM(o.order_value),2) AS total
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        GROUP BY c.customer_name
        ORDER BY total DESC
        LIMIT 10'''
}

def runQuery(queryKey, params=()):
    """Executa uma consulta SQL e retorna o resultado como um DataFrame.

    Args:
        queryKey (str): Chave da consulta no dicionário `dictQuery`.
        params (tuple, optional): Parâmetros para a consulta SQL. Padrão é ().

    Returns:
        pandas.DataFrame: DataFrame com o resultado da consulta.
    """
    query = dictQuery[queryKey]  # Obtém a consulta SQL do dicionário
    return pd.read_sql(query, conn, params=params)  # Executa a consulta e retorna o resultado como um DataFrame

Exploratory Data Analysis 🏗️📊
Analysis

In [ ]:
# Executa as consultas SQL e armazena os resultados em DataFrames
tCustomers = runQuery("queryCustomers")
tOrders = runQuery("queryOrders")
tProducts = runQuery("queryProducts")
tCategories = runQuery("queryCategories")
tOrderItens = runQuery("queryOrderItens")
tCountProducts = runQuery("queryCountProducts")
tCountCategories = runQuery("queryCountCategories")
tAvgSales = runQuery("queryAvgSales")
tDistinctCategories = runQuery("queryDistinctCategories")
tProductsByCategories = runQuery("queryProductsByCategories")
tTop10CustomerValue = runQuery("queryTop10CustomerValue")

def fig_to_base64(fig):
    """Converte uma figura Matplotlib em uma string base64 para ser exibida em HTML.

    Args:
        fig (matplotlib.figure.Figure): A figura Matplotlib a ser convertida.

    Returns:
        str: Uma string base64 representando a imagem da figura.
    """
    buf = io.BytesIO()  # Cria um buffer na memória
    fig.savefig(buf, format="png", bbox_inches='tight')  # Salva a figura no buffer como PNG
    plt.close(fig)  # Fecha a figura para liberar memória
    return base64.b64encode(buf.getvalue()).decode()  # Codifica a imagem em base64 e retorna a string

# Cria o primeiro gráfico (Quantidade de Produtos por Categoria)
fig1 = plt.figure(figsize=(8, 4))
plt.bar(tProductsByCategories['category_name'], tProductsByCategories['count'], color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.title('Quantidade de Produtos por Categoria')
plt.tight_layout()
img1 = fig_to_base64(fig1)  # Converte o gráfico em uma string base64

# Cria o segundo gráfico (Top 10 Clientes por Valor Total de Compras)
fig2 = plt.figure(figsize=(6, 6))
plt.bar(tTop10CustomerValue['customer_name'], tTop10CustomerValue['total'], color='green')
plt.title('Top 10 Clientes por Valor Total de Compras')
plt.tight_layout()
img2 = fig_to_base64(fig2)  # Converte o gráfico em uma string base64

app = dash.Dash(__name__)  # Cria uma instância do aplicativo Dash

# Define o layout do aplicativo Dash
app.layout = html.Div([
    html.H1("Análise Exploratória de Dados - E-commerce", style={"textAlign": "center", "marginBottom": "40px"}),  # Define o título principal

    html.Div([
        html.H2("Visão Geral das Tabelas", style={"marginTop": "30px"}),  # Define um subtítulo

        html.Div([
            html.H4("Clientes"),  # Define um subtítulo
            dash_table.DataTable(  # Cria uma tabela Dash
                id='customers-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tCustomers.columns],  # Define as colunas da tabela
                data=tCustomers.to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left'}  # Define o estilo das células
            )
        ], style={'marginBottom': '20px'}),  # Define o estilo do contêiner

        html.Div([
            html.H4("Pedidos"),  # Define um subtítulo
            dash_table.DataTable(  # Cria uma tabela Dash
                id='orders-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tOrders.columns],  # Define as colunas da tabela
                data=tOrders.to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left'}  # Define o estilo das células
            )
        ], style={'marginBottom': '20px'}),  # Define o estilo do contêiner

        html.Div([
            html.H4("Produtos"),  # Define um subtítulo
            dash_table.DataTable(  # Cria uma tabela Dash
                id='products-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tProducts.columns],  # Define as colunas da tabela
                data=tProducts.to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left'}  # Define o estilo das células
            )
        ], style={'marginBottom': '20px'}),  # Define o estilo do contêiner

        html.Div([
            html.H4("Categorias"),  # Define um subtítulo
            dash_table.DataTable(  # Cria uma tabela Dash
                id='categories-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tCategories.columns],  # Define as colunas da tabela
                data=tCategories.to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left'}  # Define o estilo das células
            )
        ], style={'marginBottom': '20px'}),  # Define o estilo do contêiner
    ]),

    html.H2("Métricas e Agregações", style={"marginTop": "50px"}),  # Define um subtítulo

    html.Div([
        html.H4("Contagem de Produtos"),  # Define um subtítulo
        dash_table.DataTable(  # Cria uma tabela Dash
            id='count-products-table',  # Define o ID da tabela
            columns=[{"name": col, "id": col} for col in tCountProducts.columns],  # Define as colunas da tabela
            data=tCountProducts.to_dict('records'),  # Define os dados da tabela
            style_table={'overflowX': 'auto'},  # Define o estilo da tabela
            style_cell={'textAlign': 'left'}  # Define o estilo das células
        )
    ], style={'marginBottom': '20px'}),  # Define o estilo do contêiner

    html.Div([
        html.H4("Média de Vendas"),  # Define um subtítulo
        dash_table.DataTable(  # Cria uma tabela Dash
            id='avg-sales-table',  # Define o ID da tabela
            columns=[{"name": col, "id": col} for col in tAvgSales.columns],  # Define as colunas da tabela
            data=tAvgSales.to_dict('records'),  # Define os dados da tabela
            style_table={'overflowX': 'auto'},  # Define o estilo da tabela
            style_cell={'textAlign': 'left'}  # Define o estilo das células
        )
    ], style={'marginBottom': '20px'}),  # Define o estilo do contêiner

    html.H2("Visualizações Gráficas", style={"marginTop": "50px"}),  # Define um subtítulo

    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(img1), style={"width": "100%", "maxWidth": "700px"})  # Exibe o primeiro gráfico
    ], style={'textAlign': 'center', 'marginBottom': '40px'}),  # Define o estilo do contêiner

    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(img2), style={"width": "100%", "maxWidth": "500px"})  # Exibe o segundo gráfico
    ], style={'textAlign': 'center', 'marginBottom': '40px'})  # Define o estilo do contêiner
])

if __name__ == '__main__':  # Garante que o código dentro deste bloco só seja executado quando o script for executado diretamente
    app.run(debug=True)  # Inicia o aplicativo Dash no modo de depuração

conn.close()  # Fecha a conexão com o banco de dados